In [2]:
# -*- coding: utf-8 -*-

#標準ライブラリ
import time
import re
import os
import datetime
from xml.etree import ElementTree as etree
import json

#サードパーティ(要インストール)
import pandas as pd
import numpy as np
import lxml.html
import requests as req
from concurrent import futures

In [3]:
#html型式から変換する関数
def transform_html(url):
    html_data = req.get(url)
    converted_data = lxml.html.fromstring(html_data.content)
    
    return converted_data

In [ ]:
#ページ数計算
def get_last_page(url):
    url_sum_xpath = './/*[@id="container"]/div[15]/div[5]/div/div[5]/div[2]/div/span[4]/strong/text()' #求人広告総数
    page_url_xpath = 100 #１ページあたりの
    converted_data = transform_html(url)

    page_url = converted_data.findall(page_url_xpath)
    url_sum = converted_data.xpath(url_sum_xpath)
    url_sum = url_sum[0]  #サイトによって変わる
    #url_sum = url_sum.replace(',','')

    page_num = int(url_sum)//len(page_url)+1 #あまりがでるので小数点以下切り捨てして１ページ分追加


    return page_num

In [82]:
#商品カテゴリーごとのURLの取得
ROOT_URL = 'https://www.sej.co.jp'
use_url = 'https://www.sej.co.jp/products/'
# cat_list_tmp = []
cat_list = []
cat_xpath = './/ul[@class="sideCategoryNav"]/li/a'

converted_data = transform_html(use_url)
for element in converted_data.findall(cat_xpath):
    elems = element.get('itemef')
    tmp = ''.join([ROOT_URL,elems])
    if '/a/' in tmp:
        if ('nextweek' in tmp) or ('donut' in tmp) or ('sweets' in tmp) or ('ice_cream' in tmp) or ('hotsnack' in tmp) or ('7premium' in tmp): 
            pass
        else:
            cat_list.append(tmp)


print(cat_list)

['https://www.sej.co.jp/products/a/onigiri/', 'https://www.sej.co.jp/products/a/sushi/', 'https://www.sej.co.jp/products/a/bento/', 'https://www.sej.co.jp/products/a/sandwich/', 'https://www.sej.co.jp/products/a/bread/', 'https://www.sej.co.jp/products/a/men/', 'https://www.sej.co.jp/products/a/pasta/', 'https://www.sej.co.jp/products/a/gratin/', 'https://www.sej.co.jp/products/a/dailydish/', 'https://www.sej.co.jp/products/a/salad/', 'https://www.sej.co.jp/products/a/oden/', 'https://www.sej.co.jp/products/a/chukaman/']


In [53]:
cat_list

['https://www.sej.co.jp/products/a/onigiri/',
 'https://www.sej.co.jp/products/a/sushi/',
 'https://www.sej.co.jp/products/a/bento/',
 'https://www.sej.co.jp/products/a/sandwich/',
 'https://www.sej.co.jp/products/a/bread/',
 'https://www.sej.co.jp/products/a/men/',
 'https://www.sej.co.jp/products/a/pasta/',
 'https://www.sej.co.jp/products/a/gratin/',
 'https://www.sej.co.jp/products/a/dailydish/',
 'https://www.sej.co.jp/products/a/salad/',
 'https://www.sej.co.jp/products/a/oden/',
 'https://www.sej.co.jp/products/a/chukaman/']

In [83]:
#カテゴリのリンクからもしまたリンクがあれば取得する
cat_cat_list = []

brn = ['https://www.sej.co.jp/products/a/onigiri/','https://www.sej.co.jp/products/a/sushi/']
yel = ['https://www.sej.co.jp/products/a/bread/']
blu = ['https://www.sej.co.jp/products/a/men/']
non = ['https://www.sej.co.jp/products/a/salad/', 'https://www.sej.co.jp/products/a/bento/']

brn_xpath = './/div[@class="list_btn brn pbNested pbNestedWrapper "]/div/div/a'
yel_xpath = './/div[@class="list_btn yel pbNested pbNestedWrapper "]/div/div/div/div/a'   
blu_xpath = './/div[@class="list_btn blu pbNested pbNestedWrapper "]/div/div/div/div/a'
non_xpath = './/div[@class="list_btn pbNested pbNestedWrapper "]/div/div/div/div/a'              

for x in brn:
    converted_data = transform_html(x)
    for element in converted_data.findall(brn_xpath):
        # elems = element.get('itemef')
        cat_cat_list.append(''.join([ROOT_URL,element.get('itemef')]))
for x in yel:
    converted_data = transform_html(x)
    for element in converted_data.findall(yel_xpath):
        # elems = element.get('itemef')
        cat_cat_list.append(''.join([ROOT_URL,element.get('itemef')]))
for x in blu:
    converted_data = transform_html(x)
    for element in converted_data.findall(blu_xpath):
        # elems = element.get('itemef')
        cat_cat_list.append(''.join([ROOT_URL,element.get('itemef')]))
for x in non:
    converted_data = transform_html(x)
    for element in converted_data.findall(non_xpath):
        # elems = element.get('itemef')
        cat_cat_list.append(''.join([ROOT_URL,element.get('itemef')]))

cat_cat_list
 

['https://www.sej.co.jp/products/a/cat/010010010000000/',
 'https://www.sej.co.jp/products/a/cat/010010020000000/',
 'https://www.sej.co.jp/products/a/cat/010010030000000/',
 'https://www.sej.co.jp/products/a/cat/010030010000000/',
 'https://www.sej.co.jp/products/a/cat/010030020000000/',
 'https://www.sej.co.jp/products/a/cat/010030030000000/',
 'https://www.sej.co.jp/products/a/cat/050010010000000/',
 'https://www.sej.co.jp/products/a/cat/050010020000000/',
 'https://www.sej.co.jp/products/a/cat/050020010000000/',
 'https://www.sej.co.jp/products/a/cat/050020020000000/',
 'https://www.sej.co.jp/products/a/cat/030010010000000/',
 'https://www.sej.co.jp/products/a/cat/030010020000000/',
 'https://www.sej.co.jp/products/a/cat/030010030000000/',
 'https://www.sej.co.jp/products/a/cat/030010040000000/',
 'https://www.sej.co.jp/products/a/cat/040020010000000/',
 'https://www.sej.co.jp/products/a/cat/040020020000000/',
 'https://www.sej.co.jp/products/a/cat/040020030000000/',
 'https://www.

In [84]:
cat_list.remove('https://www.sej.co.jp/products/a/onigiri/')
cat_list.remove('https://www.sej.co.jp/products/a/sushi/')
cat_list.remove('https://www.sej.co.jp/products/a/bread/')
cat_list.remove('https://www.sej.co.jp/products/a/men/')
cat_list.remove('https://www.sej.co.jp/products/a/salad/')
cat_list.remove('https://www.sej.co.jp/products/a/bento/')

In [85]:
cat_list_comp = cat_list + cat_cat_list

In [86]:
cat_list_comp

['https://www.sej.co.jp/products/a/sandwich/',
 'https://www.sej.co.jp/products/a/pasta/',
 'https://www.sej.co.jp/products/a/gratin/',
 'https://www.sej.co.jp/products/a/dailydish/',
 'https://www.sej.co.jp/products/a/oden/',
 'https://www.sej.co.jp/products/a/chukaman/',
 'https://www.sej.co.jp/products/a/cat/010010010000000/',
 'https://www.sej.co.jp/products/a/cat/010010020000000/',
 'https://www.sej.co.jp/products/a/cat/010010030000000/',
 'https://www.sej.co.jp/products/a/cat/010030010000000/',
 'https://www.sej.co.jp/products/a/cat/010030020000000/',
 'https://www.sej.co.jp/products/a/cat/010030030000000/',
 'https://www.sej.co.jp/products/a/cat/050010010000000/',
 'https://www.sej.co.jp/products/a/cat/050010020000000/',
 'https://www.sej.co.jp/products/a/cat/050020010000000/',
 'https://www.sej.co.jp/products/a/cat/050020020000000/',
 'https://www.sej.co.jp/products/a/cat/030010010000000/',
 'https://www.sej.co.jp/products/a/cat/030010020000000/',
 'https://www.sej.co.jp/produc

In [98]:
converted_data = transform_html('https://www.sej.co.jp/products/a/sandwich/')
for x in converted_data.findall('.//div[@class="item_ttl"]/p/a'):
    print(x)

<Element a at 0x116db47c0>
<Element a at 0x116db4680>
<Element a at 0x115daab80>
<Element a at 0x11697e860>
<Element a at 0x11697e4a0>
<Element a at 0x116de5810>
<Element a at 0x116de5b80>
<Element a at 0x11697e9f0>
<Element a at 0x116a396d0>
<Element a at 0x116a39720>
<Element a at 0x116a394f0>
<Element a at 0x116db4f90>
<Element a at 0x115d35360>
<Element a at 0x115d35540>
<Element a at 0x115d35590>


In [100]:
param = 'page/l100'
url_xpath = './/div[@class="item_ttl"]/p/a'
url_list = []

for x in cat_list_comp:
# for x in ['https://www.sej.co.jp/products/a/sandwich/']:
    for y in range(1,4):
        param = '{}/l100'.format(y)
        combined_url = ''.join([x,param])
        converted_data = transform_html(combined_url)
        # print(converted_data)
        for element in converted_data.findall(url_xpath):
        # # elems = element.get('itemef')
            url_list.append(''.join([ROOT_URL,element.get('itemef')]))
        time.sleep(1)
    print('finish:'+ x)
url_list



finish:https://www.sej.co.jp/products/a/sandwich/
finish:https://www.sej.co.jp/products/a/pasta/
finish:https://www.sej.co.jp/products/a/gratin/
finish:https://www.sej.co.jp/products/a/dailydish/
finish:https://www.sej.co.jp/products/a/oden/
finish:https://www.sej.co.jp/products/a/chukaman/
finish:https://www.sej.co.jp/products/a/cat/010010010000000/
finish:https://www.sej.co.jp/products/a/cat/010010020000000/
finish:https://www.sej.co.jp/products/a/cat/010010030000000/
finish:https://www.sej.co.jp/products/a/cat/010030010000000/
finish:https://www.sej.co.jp/products/a/cat/010030020000000/
finish:https://www.sej.co.jp/products/a/cat/010030030000000/
finish:https://www.sej.co.jp/products/a/cat/050010010000000/
finish:https://www.sej.co.jp/products/a/cat/050010020000000/
finish:https://www.sej.co.jp/products/a/cat/050020010000000/
finish:https://www.sej.co.jp/products/a/cat/050020020000000/
finish:https://www.sej.co.jp/products/a/cat/030010010000000/
finish:https://www.sej.co.jp/products

['https://www.sej.co.jp/products/a/item/051550/',
 'https://www.sej.co.jp/products/a/item/051545/',
 'https://www.sej.co.jp/products/a/item/050516/',
 'https://www.sej.co.jp/products/a/item/051402/',
 'https://www.sej.co.jp/products/a/item/051555/',
 'https://www.sej.co.jp/products/a/item/050510/',
 'https://www.sej.co.jp/products/a/item/051547/',
 'https://www.sej.co.jp/products/a/item/050367/',
 'https://www.sej.co.jp/products/a/item/051508/',
 'https://www.sej.co.jp/products/a/item/051503/',
 'https://www.sej.co.jp/products/a/item/051463/',
 'https://www.sej.co.jp/products/a/item/051160/',
 'https://www.sej.co.jp/products/a/item/051457/',
 'https://www.sej.co.jp/products/a/item/051456/',
 'https://www.sej.co.jp/products/a/item/051449/',
 'https://www.sej.co.jp/products/a/item/051445/',
 'https://www.sej.co.jp/products/a/item/051427/',
 'https://www.sej.co.jp/products/a/item/051426/',
 'https://www.sej.co.jp/products/a/item/051146/',
 'https://www.sej.co.jp/products/a/item/051512/',


In [101]:
len(url_list)

1498

In [102]:
url_df = pd.DataFrame(url_list,columns=["url"])
url_df.to_json('seven_url.json',force_ascii=False)

In [ ]:
url_df= pd.read_json('seven_url.json')

In [142]:
'''
取得したURLから求人広告の概要を取得するための関数
必要ないデータもまとめて取得してあとからいらないものを削除するようにしています
'''
def get_data(i):

    item_dict = {}
    url = url_df['url'][i]
    converted_data = transform_html(url)

    name = converted_data.xpath('.//div[@class="item_ttl"]/h1/text()')
    item_dict.setdefault('商品名',name)
    
    price_tmp = converted_data.findall('.//div[@class="item_price"]/p')
    price =''
    for x in price_tmp:
        price+=x.text
    item_dict.setdefault('値段',price)
    
    text_tmp = converted_data.findall('.//div[@class="item_text"]/p')
    text =''
    for x in text_tmp:
        text+=x.text
    item_dict.setdefault('説明',text)
    
    job_requirement_th_xpath = './/table/tbody/tr/th' #辞書のキーにする部分(カラム名)
    job_requirement_th = converted_data.findall(job_requirement_th_xpath)

    job_requirement_td_xpath = './/table/tbody/tr/td' #辞書のバリューにする部分（店ごとのデータ）
    job_requirement_td = converted_data.findall(job_requirement_td_xpath)

    for job_requirement_th, job_requirement_td in zip(job_requirement_th, job_requirement_td):
        job_requirement_td_text = ''
        job_requirement_th_text = job_requirement_th.text
        job_requirement_td_texts = job_requirement_td.itertext() #バリューの指定したxpathの子階層のテキストも全取得する
        for td_text in job_requirement_td_texts:
            job_requirement_td_text += td_text
        item_dict.setdefault(job_requirement_th_text, job_requirement_td_text)

    item_dict.setdefault('URL', url)
    item_dict.setdefault('店名', 'セブンイレブン')

    # time.sleep(2)
    print(i)

    return item_dict

In [135]:
get_data(0)

{'商品名': ['ももももサンド'],
 '値段': '340円（税込367.20円）',
 '説明': 'カスタードクリームとホイップクリームを組み合わせ、２種類の桃を使用したサンドイッチです。',
 '本製品に含まれるアレルギー物質': '\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t卵・乳・小麦\t\t\t\t\t\n\t\t\t\t',
 '栄養成分': '熱量：320kcal、たんぱく質：4.1g、脂質：17.3g、炭水化物：37.9g（糖質：35.8g、食物繊維：2.1g）、食塩相当量：0.47g',
 'URL': 'https://www.sej.co.jp/products/a/item/051550/',
 '店名': 'セブンイレブン'}

In [137]:
from tqdm import tqdm

In [143]:
%%time
'''
並行処理で取得していく
joblibも使用したことがあるがこちらのほうが止まることなくスムーズに処理ができた
'''
myThreads = []
with futures.ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(0,1500):
        try: 
            myThreads.append(executor.submit(get_data,i))
        except:
            print('error')
            continue
        

result = []
for x in myThreads:
    try:
        item = x.result()
        result.append(item)
    except:
        pass

df = pd.DataFrame(result)
df

3
2
1
4
6
8
0
5
7
9
10
13
14
11
12
16
17
18
19
21
23
24
15
26
20
22
25
27
28
31
29
34
30
36
32
37
33
39
40
41
35
43
45
38
42
48
44
50
46
47
51
53
49
52
54
55
57
58
60
56
63
59
61
65
66
62
64
67
70
69
68
72
75
76
73
71
74
78
81
77
82
83
79
80
87
88
84
89
90
86
85
92
91
94
93
95
96
97
98
100
101
104
105
99
106
102
107
108
103
110
111
114
115
116
109
112
118
113
117
122
119
121
120
125
124
127
126
123
129
130
131
128
132
135
133
134
136
137
140
142
138
139
141
144
147
146
143
149
151
145
152
148
150
155
157
156
153
159
154
160
163
158
161
162
167
164
165
169
170
166
168
171
174
172
176
173
178
177
175
179
182
184
180
185
186
188
181
183190

187
192
189
191
193
196
194
199
195
201
198
197
200
204
202
206
207
203
210
205
208
209
214
212
215
211
218
216
213
217
219
221
222
220
223
226
227
225
224
230
231
228
229
233
234
236
237
232
235
238
239
243
241
240
244
245
247
248
246
242
250
252
253
255
256
249
251
259
257
254
262
263
258
264
261
260
265
266
267
271
268
272
269
270
275
273
277
274
28

,商品名,値段,説明,本製品に含まれるアレルギー物質,栄養成分,URL,店名,主要原料・主要原料原産地,原産国
0,[ももももサンド],340円（税込367.20円）,カスタードクリームとホイップクリームを組み合わせ、２種類の桃を使用したサンドイッチです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：320kcal、たんぱく質：4.1g、脂質：17.3g、炭水化物：37.9g（糖質：3...,https://www.sej.co.jp/products/a/item/051550/,セブンイレブン,NaN,NaN
1,[ももももサンド],340円（税込367.20円）,カスタードクリームとホイップクリームを組み合わせ、２種類の桃を使用したサンドイッチです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：306kcal、たんぱく質：4.1g、脂質：16.0g、炭水化物：37.4g（糖質：3...,https://www.sej.co.jp/products/a/item/051545/,セブンイレブン,NaN,NaN
2,[シャインマスカット入りフルーツサンド],390円（税込421.20円）,９月～１０月に旬を迎えるマスカットを使用したフルーツサンド。他にもパイン、黄桃、みかんの３種...,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：260kcal、たんぱく質：3.5g、脂質：12.8g、炭水化物：33.4g（糖質：3...,https://www.sej.co.jp/products/a/item/050516/,セブンイレブン,NaN,NaN
3,[大きなソーセージ チリドッグ],310円（税込334.80円）,パリッとジューシーなソーセージにスパイシーなチリソースとチーズを盛り付けました。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：478kcal、たんぱく質：18.8g、脂質：34.0g、炭水化物：25.1g（糖質：...,https://www.sej.co.jp/products/a/item/051402/,セブンイレブン,NaN,NaN
4,[てりたまチーズバーガー],380円（税込410.40円）,ビーフハンバーグと潰し目玉焼きとチェダーチーズを組み合わせ、照焼ソースとオーロラソースの２種...,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：511kcal、たんぱく質：24.1g、脂質：25.6g、炭水化物：47.9g（糖質：...,https://www.sej.co.jp/products/a/item/051555/,セブンイレブン,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1493,[７プレミアムゼスプリグリーンキウイ],188円（税込203.04円）,,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：37kcal、たんぱく質：0.6g、脂質：0.4g、炭水化物：9.3g（糖質：7.7g...,https://www.sej.co.jp/products/a/item/250998/,セブンイレブン,NaN,NaN
1494,[ちょこっとカットパイン],195円（税込210.60円）,,NaN,NaN,https://www.sej.co.jp/products/a/item/251026/,セブンイレブン,NaN,NaN
1495,[７プレミアム干しいも],338円（税込365.04円）,ほど良い食感と素朴な甘さが特長の干しいもです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：260kcal、たんぱく質：3.9g、脂質：0.5g、炭水化物：62.4g、食塩相当量...,https://www.sej.co.jp/products/a/item/250735/,セブンイレブン,NaN,NaN
1496,[７プレミアム けんちん汁・豚汁用水煮],278円（税込300.24円）,下ごしらえいらず 家庭の味をちょっとお手伝い,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：26kcal、たんぱく質：0.6g、脂質：0.1g、炭水化物：6.1g（糖質：3.4g...,https://www.sej.co.jp/products/a/item/250594/,セブンイレブン,NaN,NaN


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   商品名              1498 non-null   object
 1   値段               1498 non-null   object
 2   説明               1498 non-null   object
 3   本製品に含まれるアレルギー物質  1474 non-null   object
 4   栄養成分             1474 non-null   object
 5   URL              1498 non-null   object
 6   店名               1498 non-null   object
 7   主要原料・主要原料原産地     41 non-null     object
 8   原産国              41 non-null     object
dtypes: object(9)
memory usage: 105.5+ KB


In [145]:
df.to_json('seven_data.json',force_ascii=False)